In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium 
from folium import features
import vincent
import os
import json

In [25]:

t_inf=1
t_gue=0.5
t_mort=0
d_t = 1
nb_pas = 2000

In [26]:
class Pays:
    def __init__(self):
        self.popt = 100
        self.popinf = 10
        self.pops = self.popt - self.popinf
        self.popimu=0
    
#calcul de la probabilité d'infection
def prob_inf(pays):
    return (t_inf * pays.popinf * pays.pops * d_t) / (pays.popt ** 2)

#calcul de la probabilité de guérison
def prob_gue(pays):
    return (t_gue * pays.popinf * d_t) / pays.popt

#la simulation se fait en posant comme hypothèse que d_t est assez petit pour qu'il y ait à chaque transition au plus un changement d'état
def evolution():
    france = Pays()
    i = 1
    tab = []
    while (i < nb_pas):
        tab.append(france.popinf)
        x = np.random.uniform()
        if 0 <= x and x < prob_inf(france):
            france.popinf = france.popinf + 1
            france.pops=france.pops-1
        elif prob_inf(france) <= x and x < prob_inf(france) + prob_gue(france):
            france.popimu = france.popimu + 1
            france.popinf = france.popinf - 1
        else:
            pass

        i = i + 1
    return tab




In [27]:
#ajout de la carte
regions=os.path.join('world-countries.json')

In [28]:
#ajout des données
donnees=pd.read_csv(os.path.join('population_par_pays.csv'))

In [29]:
#initialisiation de la carte
m=folium.Map(location=[0, 0], zoom_start=1.2)

In [30]:
np.max(donnees["pop"])

1378665000.0

In [31]:
m.choropleth(
 geo_data=regions,
 name='visualisation',
 data=donnees,
 columns=['Country Code','pop'],
 key_on='feature.id',
 fill_color='PuRd',
 fill_opacity=0.7,
 line_opacity=0.2,
legend_name='population infectée (%)'
)

#ajout du popup à chaque pays
popup=folium.Popup()
a=folium.GeoJson(regions,name='pays')
m.add_child(a.add_child(popup))

folium.LayerControl().add_to(m)


In [32]:
d=dict()
t=0
for i in evolution():
    d[t]= i
    t=t+1


In [33]:

line = vincent.Line(d, height=300, width=300)
line.axis_titles(x='temps', y='pop. infectée')
data = json.loads(line.to_json())

In [34]:
v = features.Vega(data, width='100%', height='100%')
v.add_to(popup)

In [35]:
m